In [3]:
import pandas as pd
data = pd.read_csv(r'DataSet\all_kindle_review.csv', encoding='latin-1')

data.head()

,Unnamed: 0.1,Unnamed: 0,asin,helpful,rating,reviewText,reviewTime,reviewerID,reviewerName,summary,unixReviewTime
0,0,11539,B0033UV8HI,"[8, 10]",3,"Jace Rankin may be short, but he's nothing to ...","09 2, 2010",A3HHXRELK8BHQG,Ridley,Entertaining But Average,1283385600
1,1,5957,B002HJV4DE,"[1, 1]",5,Great short read. I didn't want to put it dow...,"10 8, 2013",A2RGNZ0TRF578I,Holly Butler,Terrific menage scenes!,1381190400
2,2,9146,B002ZG96I4,"[0, 0]",3,I'll start by saying this is the first of four...,"04 11, 2014",A3S0H2HV6U1I7F,Merissa,Snapdragon Alley,1397174400
3,3,7038,B002QHWOEU,"[1, 3]",3,Aggie is Angela Lansbury who carries pocketboo...,"07 5, 2014",AC4OQW3GZ919J,Cleargrace,very light murder cozy,1404518400
4,4,1776,B001A06VJ8,"[0, 1]",4,I did not expect this type of book to be in li...,"12 31, 2012",A3C9V987IQHOQD,Rjostler,Book,1356912000


In [4]:
df=data[['reviewText','rating']]
df.head()

,reviewText,rating
0,"Jace Rankin may be short, but he's nothing to ...",3
1,Great short read. I didn't want to put it dow...,5
2,I'll start by saying this is the first of four...,3
3,Aggie is Angela Lansbury who carries pocketboo...,3
4,I did not expect this type of book to be in li...,4


In [6]:
df.isnull().sum()

reviewText    0
rating        0
dtype: int64

In [7]:
df['rating'].unique()

array([3, 5, 4, 2, 1], dtype=int64)

In [8]:
df['rating'].value_counts()

rating
5    3000
4    3000
3    2000
2    2000
1    2000
Name: count, dtype: int64

In [9]:
## postive review is 1 and negative review is 0
df['rating']=df['rating'].apply(lambda x:0 if x<3 else 1)

C:\Users\dinka\AppData\Local\Temp\ipykernel_17240\3208188810.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['rating']=df['rating'].apply(lambda x:0 if x<3 else 1)


In [10]:
df['reviewText']=df['reviewText'].str.lower()

C:\Users\dinka\AppData\Local\Temp\ipykernel_17240\2784960931.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['reviewText']=df['reviewText'].str.lower()


In [11]:
df.head(3)

,reviewText,rating
0,"jace rankin may be short, but he's nothing to ...",1
1,great short read. i didn't want to put it dow...,1
2,i'll start by saying this is the first of four...,1


In [12]:
import re
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\dinka\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [13]:
from bs4 import BeautifulSoup

In [30]:
import re
from bs4 import BeautifulSoup
from nltk.corpus import stopwords

# Make sure df is a copy
df = data[['reviewText']].copy()

# Remove special characters
df.loc[:, 'reviewText'] = df['reviewText'].apply(lambda x: re.sub('[^a-z A-Z 0-9-]+', '', str(x)))

# Remove stopwords
df.loc[:, 'reviewText'] = df['reviewText'].apply(
    lambda x: " ".join([y for y in x.split() if y.lower() not in stopwords.words('english')])
)

# Remove URLs
df.loc[:, 'reviewText'] = df['reviewText'].apply(
    lambda x: re.sub(r'(http|https|ftp|ssh)://\S+', '', str(x))
)

# Remove HTML tags (use lxml if installed, else fallback to html.parser)
df.loc[:, 'reviewText'] = df['reviewText'].apply(lambda x: BeautifulSoup(x, 'html.parser').get_text())

# Remove extra spaces
df.loc[:, 'reviewText'] = df['reviewText'].apply(lambda x: " ".join(x.split()))


In [35]:
from nltk.stem import WordNetLemmatizer

In [36]:
lemmatizer=WordNetLemmatizer()

In [37]:
def lemmatize_words(text):
    return " ".join([lemmatizer.lemmatize(word) for word in text.split()])

In [38]:
df['reviewText']=df['reviewText'].apply(lambda x:lemmatize_words(x))

In [42]:
print(df.columns.tolist())
print(df.head())


['reviewText']
                                          reviewText
0  Jace Rankin may short he nothing mess man haul...
1  Great short read didnt want put read one sitti...
2  Ill start saying first four book wasnt expecti...
3  Aggie Angela Lansbury carry pocketbook instead...
4  expect type book library pleased find price right


In [40]:
## Train Test Split
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(
    df['reviewText'], 
    df['overall'], 
    test_size=0.20, 
    random_state=42
)


KeyError: 'overall'

In [20]:
from sklearn.feature_extraction.text import CountVectorizer
bow=CountVectorizer()
X_train_bow=bow.fit_transform(X_train).toarray()
X_test_bow=bow.transform(X_test).toarray()

In [21]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf=TfidfVectorizer()
X_train_tfidf=tfidf.fit_transform(X_train).toarray()
X_test_tfidf=tfidf.transform(X_test).toarray()

In [22]:
from sklearn.naive_bayes import GaussianNB
nb_model_bow=GaussianNB().fit(X_train_bow,y_train)
nb_model_tfidf=GaussianNB().fit(X_train_tfidf,y_train)

In [23]:
from sklearn.metrics import confusion_matrix,accuracy_score,classification_report

In [24]:
y_pred_bow=nb_model_bow.predict(X_test_bow)

In [25]:
y_pred_tfidf=nb_model_bow.predict(X_test_tfidf)

In [26]:
confusion_matrix(y_test,y_pred_bow)

array([[495, 285],
       [731, 889]], dtype=int64)

In [27]:
print("BOW accuracy: ",accuracy_score(y_test,y_pred_bow))

BOW accuracy:  0.5766666666666667


In [28]:
confusion_matrix(y_test,y_pred_tfidf)

array([[480, 300],
       [726, 894]], dtype=int64)

In [29]:
print("TFIDF accuracy: ",accuracy_score(y_test,y_pred_tfidf))

TFIDF accuracy:  0.5725
